## IMPORT NECESARIOS

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, desc, asc


## CREACION DE LA SESION DE SPARK

In [2]:
spark = SparkSession.builder\
    .appName("ETL meteoritos")\
    .getOrCreate()


In [3]:
spark

## LECTURA DE LOS DATOS

In [5]:
import os
print(os.path.exists("/home/jovyan/work/Meteorite.csv"))


False


In [6]:
!ls -lah /home/jovyan/work/


total 3.8M
drwxrwxr-x 4 jovyan  1000 4.0K Mar  2 19:14 .
drwsrws--- 1 jovyan users 4.0K Mar  2 18:51 ..
-rw-rw-r-- 1 jovyan  1000  18K Mar  2 19:14 etl_meteoritos.ipynb
drwxrwxr-x 2 jovyan  1000 4.0K Dec 13  1901 iframe_figures
drwxrwxr-x 2 jovyan  1000 4.0K Mar  2 19:12 .ipynb_checkpoints
-rw-rw-r-- 1 jovyan  1000 3.7M Jan  1  1980 m.csv
-rw-rw-r-- 1 jovyan  1000  42K Mar  2 19:12 meteoritos_eda.ipynb
-rw-r--r-- 1 jovyan users  617 Mar  2 19:14 Untitled.ipynb


In [7]:
import os
print(os.access("/home/jovyan/work/Meteorite.csv", os.R_OK))


False


In [4]:
df= spark.read.csv("Meteorite.csv", header=True, inferSchema= True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/jovyan/work/Meteorite.csv.

In [5]:
df.show()

+-------------------+---+--------+------------+--------+----+----+---------+---------+--------------------+----+
|               name| id|nametype|    recclass|mass (g)|fall|year|   reclat|  reclong|         GeoLocation|_c10|
+-------------------+---+--------+------------+--------+----+----+---------+---------+--------------------+----+
|             Aachen|  1|   Valid|          L5|    21.0|Fell|1880|   50.775|  6.08333|   (50.775, 6.08333)|NULL|
|             Aarhus|  2|   Valid|          H6|   720.0|Fell|1951| 56.18333| 10.23333|(56.18333, 10.23333)|NULL|
|               Abee|  6|   Valid|         EH4|107000.0|Fell|1952| 54.21667|   -113.0|  (54.21667, -113.0)|NULL|
|           Acapulco| 10|   Valid| Acapulcoite|  1914.0|Fell|1976| 16.88333|    -99.9|   (16.88333, -99.9)|NULL|
|            Achiras|370|   Valid|          L6|   780.0|Fell|1902|-33.16667|   -64.95| (-33.16667, -64.95)|NULL|
|           Adhi Kot|379|   Valid|         EH4|  4239.0|Fell|1919|     32.1|     71.8|        (3

## TRANSFORMACIONES

Tenemos una columna que solo tiene nulos, y la de GeoLocation que no vamos a utilizar

In [6]:
df_es = df.withColumnsRenamed({"name":"nombre", "nametype": "condicion", "recclass": "clasificacion", "mass (g)": "masa_gramos", "fall": "tipo_descenso", "year":"año", "reclat": "latitud", "reclong":"longitud"})

Vamos a tocar los nombres de las columnas, asi cuando hagamos una visualizacion de datos se vera mas bonito

In [7]:
df_drop = df_es.drop("GeoLocation","_c10")

In [8]:
df_drop.show()

+-------------------+---+---------+-------------+-----------+-------------+----+---------+---------+
|             nombre| id|condicion|clasificacion|masa_gramos|tipo_descenso| año|  latitud| longitud|
+-------------------+---+---------+-------------+-----------+-------------+----+---------+---------+
|             Aachen|  1|    Valid|           L5|       21.0|         Fell|1880|   50.775|  6.08333|
|             Aarhus|  2|    Valid|           H6|      720.0|         Fell|1951| 56.18333| 10.23333|
|               Abee|  6|    Valid|          EH4|   107000.0|         Fell|1952| 54.21667|   -113.0|
|           Acapulco| 10|    Valid|  Acapulcoite|     1914.0|         Fell|1976| 16.88333|    -99.9|
|            Achiras|370|    Valid|           L6|      780.0|         Fell|1902|-33.16667|   -64.95|
|           Adhi Kot|379|    Valid|          EH4|     4239.0|         Fell|1919|     32.1|     71.8|
|Adzhi-Bogdo (stone)|390|    Valid|        LL3-6|      910.0|         Fell|1949| 44.83333| 

## ELIMINACION DE NULOS 

In [12]:
df_null = df_drop.dropna(subset=["latitud", "longitud"])


## OUTPUT